# Batch Simulation

### Load Libraries

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import cobra
import cplex

ModuleNotFoundError: No module named 'cplex'

In [ ]:
def EFlux2(model, Transcriptomics):
    
    mets = [m.id for m in model.metabolites]
    rxns = [r.id for r in model.reactions]
    nrow = len(mets)
    ncol = len(rxns)

    c = [r.objective_coefficient for r in model.reactions]

    gpr_dict = dict()
    for r in model.reactions:
        if r.gene_reaction_rule:
            temp = set()
            for x in [x.strip('() ') for x in r.gene_reaction_rule.split(' or ')]:
                temp.add(frozenset(y.strip('() ') for y in x.split(' and ')))
            gpr_dict[r.id] = temp

    for r in model.reactions:
        if r.lower_bound == -1000.0:
            r.lower_bound = -np.Inf
        if r.upper_bound == 1000.0:
            r.upper_bound = np.Inf

    lb = []
    ub = []
    for r in model.reactions:
        if r.gene_reaction_rule:
            t = np.sum([np.min([Transcriptomics.loc[g] if g in Transcriptomics.index 
                                else np.array([np.Inf]) for g in p])
                        for p in gpr_dict[r.id]])
            if r.lower_bound < 0.0:
                lb.append(-t)
            else:
                lb.append(r.lower_bound)
            if r.upper_bound > 0.0:
                ub.append(t)
            else:
                ub.append(r.upper_bound)
        else:
            lb.append(r.lower_bound)
            ub.append(r.upper_bound)

    EFlux2 = cplex.Cplex()
    EFlux2.set_results_stream(None)
    EFlux2.parameters.simplex.tolerances.optimality.set(1e-9)
    EFlux2.parameters.simplex.tolerances.feasibility.set(1e-9)

    EFlux2.linear_constraints.add(rhs=[0]*nrow, senses='E'*nrow, names=mets)
    EFlux2.variables.add(obj=c, lb=lb, ub=ub, names=rxns)
    for r in model.reactions:
        for m, v in r.metabolites.items():
            EFlux2.linear_constraints.set_coefficients(m.id, r.id, v)

    EFlux2.objective.set_sense(EFlux2.objective.sense.maximize)
    EFlux2.solve()
    
    Q = [1.0 for r in model.reactions]
    
    EFlux2_min = cplex.Cplex()
    EFlux2_min.set_results_stream(None)
    EFlux2_min.parameters.simplex.tolerances.optimality.set(1e-9)
    EFlux2_min.parameters.simplex.tolerances.feasibility.set(1e-9)

    EFlux2_min.linear_constraints.add(rhs=[0]*nrow, senses='E'*nrow, names=mets)
    EFlux2_min.variables.add(obj=[0]*ncol, lb=lb, ub=ub, names=rxns)
    for r in model.reactions:
        for m, v in r.metabolites.items():
            EFlux2_min.linear_constraints.set_coefficients(m.id, r.id, v)

    EFlux2_min.variables.set_lower_bounds(biomass, 1.0*EFlux2.solution.get_objective_value())
    EFlux2_min.objective.set_quadratic(Q)
    EFlux2_min.objective.set_sense(EFlux2_min.objective.sense.minimize)
    EFlux2_min.solve()
    
    sol = type('',(),{})()
    sol = pd.Series(data=EFlux2_min.solution.get_values(), index=rxns)
    sol.objective_value = (EFlux2.solution.get_objective_value(),
                           EFlux2_min.solution.get_objective_value())
    sol.status = (EFlux2.solution.get_status_string(),
                  EFlux2_min.solution.get_status_string())
    
    return(sol)

def SPOT(model, Transcriptomics):
    
    mets = [m.id for m in model.metabolites]
    rxns = [r.id for r in model.reactions]
    nrow = len(mets)
    ncol = len(rxns)

    rev_rxns = ['rev_'+r.id for r in model.reactions if r.reversibility]
    rev_ncol = len(rev_rxns)

    gpr_dict = dict()
    for r in model.reactions:
        if r.gene_reaction_rule:
            temp = set()
            for x in [x.strip('() ') for x in r.gene_reaction_rule.split(' or ')]:
                temp.add(frozenset(y.strip('() ') for y in x.split(' and ')))
            gpr_dict[r.id] = temp

    lb = [0.0 if r.reversibility else r.lower_bound for r in model.reactions] + [0.0 for r in model.reactions if r.reversibility]
    ub = [r.upper_bound for r in model.reactions] + [-r.lower_bound for r in model.reactions if r.reversibility]
        
    c = []
    for r in model.reactions:
        if r.gene_reaction_rule:
            t = np.sum([np.min([Transcriptomics.loc[g] if g in Transcriptomics.index 
                                else np.array([np.Inf]) for g in p])
                        for p in gpr_dict[r.id]])
            if t == np.Inf:
                t = 0
            c.append(t)
        else:
            c.append(0.0)
    for r in model.reactions:
        if r.reversibility:
            if r.gene_reaction_rule:
                t = np.sum([np.min([Transcriptomics.loc[g] if g in Transcriptomics.index 
                                    else np.array([np.Inf]) for g in p])
                            for p in gpr_dict[r.id]])
                if t == np.Inf:
                    t = 0
                c.append(t)
            else:
                c.append(0.0)

    SPOT = cplex.Cplex()
    SPOT.set_results_stream(None)
    SPOT.parameters.simplex.tolerances.optimality.set(1e-9)
    SPOT.parameters.simplex.tolerances.feasibility.set(1e-9)

    SPOT.linear_constraints.add(rhs=[0]*nrow, senses='E'*nrow, names=mets)
    SPOT.variables.add(obj=c, lb=lb, ub=ub, names=rxns+rev_rxns)
    for r in model.reactions:
        for m, v in r.metabolites.items():
            SPOT.linear_constraints.set_coefficients(m.id, r.id, v)
    for r in model.reactions:
        if r.reversibility:
            for m, v in r.metabolites.items():
                SPOT.linear_constraints.set_coefficients(m.id, 'rev_'+r.id, -v)
    SPOT.quadratic_constraints.add(quad_expr=[rxns+rev_rxns, rxns+rev_rxns, [1]*len(c)],
                                   sense='L', rhs=100000.0, name='L2norm')
    SPOT.objective.set_sense(SPOT.objective.sense.maximize)
    SPOT.solve()
    SPOT_sol = SPOT.solution.get_objective_value()

    sol = type('',(),{})()
    temp = pd.Series(data=SPOT.solution.get_values(), index=rxns+rev_rxns)
    flux = temp.loc[rxns]
    flux_rev = temp.loc[rev_rxns]
    for r in model.reactions:
        if r.reversibility:
            flux.loc[r.id] = flux.loc[r.id] - flux_rev.loc['rev_'+r.id]
    sol = flux
    sol.objective_value = SPOT.solution.get_objective_value()
    sol.status = SPOT.solution.get_status_string()
    
    return(sol)

### Load a metabolic model

In [ ]:
model = cobra.io.load_json_model("iML1515.json")

In [ ]:
model

In [ ]:
biomass = 'BIOMASS_Ec_iML1515_core_75p37M'

In [ ]:
model.medium

In [ ]:
model.solver

In [ ]:
model.solver = 'cplex'
sol = model.optimize()
print(sol.status, sol.objective_value)

In [ ]:
sol = cobra.flux_analysis.pfba(model)
print(sol.status, sol[biomass])

In [ ]:
Transcriptomics = pd.read_csv('ishii-trans-1.csv', index_col=0, header=None)
Transcriptomics.head()

In [ ]:
sol = EFlux2(model,Transcriptomics[1])
print(sol.status, sol.objective_value)

In [ ]:
sol = SPOT(model,Transcriptomics[1])
print(sol.status, sol[biomass], sol.objective_value)

In [ ]:
print('Uptake rates')
for x in model.exchanges:
    if sol[x.id] < -1e-6:
        print(x.id, sol[x.id])
print()
print('Secretion rates')
for x in model.exchanges:
    if sol[x.id] > 1e-6:
        print(x.id, sol[x.id])

### Define medium and initial concentrations

In [ ]:
volume = 1.0
comp = ['glc__D_e', 'nh4_e', 'pi_e', 'so4_e', 'mg2_e', 'k_e', 'na1_e', 'cl_e']
subs0 = [22.203, 18.695, 69.454, 2.0, 2.0, 21.883, 103.7, 27.25] # in mM
cell0 = 0.01 # in gDW/L
pd.DataFrame(index=comp, data=subs0, columns=['mM'])

In [ ]:
subs_ext = {r.id: r.reactants[0].id for r in model.exchanges if r.reactants[0].id in comp}
subs_ext

In [ ]:
prod_ext = {r.id: r.reactants[0].id for r in model.exchanges if r.upper_bound > 0.0}
prod_ext

### Set time interval and concentration profiles

In [ ]:
t0 = 0.0
tf = 24.0
pts = 13
tspan = np.linspace(t0, tf, pts)
delt = tspan[1] - tspan[0]
cell = pd.Series(index=tspan)
cell[t0] = cell0
subs = pd.DataFrame(index=tspan, columns=comp)
subs.loc[t0] = subs0
prod = pd.DataFrame(index=tspan, columns=prod_ext.values())
prod.loc[t0] = 0.0

### Solve pFBA at each time interval and update concentrations

In [ ]:
with model:
    for t in tspan:
        for k, v in subs_ext.items():
            model.reactions.get_by_id(k).lower_bound = max(model.reactions.get_by_id(k).lower_bound,
                                                           -subs.loc[t,v]*volume/cell[t]/delt)
        sol = model.optimize()
        #sol = cobra.flux_analysis.pfba(model)
        mu = sol[biomass]
        print(t, sol.status, mu)
        if sol.status == 'optimal' and mu > 1e-6:
            cell[t+delt] = cell[t]*np.exp(mu*delt)
            for k, v in subs_ext.items():
                subs.loc[t+delt,v] = max(subs.loc[t,v]-sol[k]/mu*cell[t]*(1-np.exp(mu*delt)),0.0)
            for k, v in prod_ext.items():
                if sol[k] > 0:
                    if v in prod.columns:
                        prod.loc[t+delt,v] = prod.loc[t,v]-sol[k]/mu*cell[t]*(1-np.exp(mu*delt))
                    else:
                        prod.loc[0:t,v] = 0
                        prod.loc[t+delt,v] = prod.loc[t,v]-sol[k]/mu*cell[t]*(1-np.exp(mu*delt))
        else:
            #break
            cell[t+delt] = cell[t]
            for k, v in subs_ext.items():
                subs.loc[t+delt,v] = subs.loc[t,v]
            for k, v in prod_ext.items():
                prod.loc[t+delt,v] = prod.loc[t,v]

t = 24
cell = cell[0:t]
subs = subs.loc[0:t]
prod = prod.loc[0:t, prod.columns[prod.max() > 0.0]]

### Plot cell mass and concentration profiles

In [ ]:
fig, ax = plt.subplots(figsize=(12,4), ncols=3, nrows=1)
cell.plot(ax=ax[0], style='s-', title='Cell', label='dcw', legend=True)
ax[0].set_xlabel("Hour")
ax[0].set_ylabel("Concentration (gDW/L)")
subs.plot(ax=ax[1], style='o-', title='Substrates')
ax[1].set_xlabel("Hour")
ax[1].set_ylabel("Concentration (mM)")
prod.plot(ax=ax[2], style='o-', title='Products')
ax[2].set_xlabel("Hour")
ax[2].set_ylabel("Concentration (mM)")
plt.tight_layout()

### Set time interval and concentration profiles

In [ ]:
t0 = 0.0
tf = 24.0
pts = 13
tspan = np.linspace(t0, tf, pts)
delt = tspan[1] - tspan[0]
cell = pd.Series(index=tspan)
cell[t0] = cell0
subs = pd.DataFrame(index=tspan, columns=comp)
subs.loc[t0] = subs0
prod = pd.DataFrame(index=tspan, columns=prod_ext.values())
prod.loc[t0] = 0.0

### Solve EFlux2 at each time interval and update concentrations

In [ ]:
with model:
    for t in tspan:
        for k, v in subs_ext.items():
            model.reactions.get_by_id(k).lower_bound = max(model.reactions.get_by_id(k).lower_bound,
                                                           -subs.loc[t,v]*volume/cell[t]/delt)
        sol = EFlux2(model,Transcriptomics[1])
        #sol = SPOT(model,Transcriptomics[t])
        mu = sol[biomass]
        print(t, sol.status, mu)
        if sol.status == ('optimal','optimal') and mu > 1e-6:
            cell[t+delt] = cell[t]*np.exp(mu*delt)
            for k, v in subs_ext.items():
                subs.loc[t+delt,v] = max(subs.loc[t,v]-sol[k]/mu*cell[t]*(1-np.exp(mu*delt)),0.0)
            for k, v in prod_ext.items():
                if sol[k] > 0:
                    if v in prod.columns:
                        prod.loc[t+delt,v] = prod.loc[t,v]-sol[k]/mu*cell[t]*(1-np.exp(mu*delt))
                    else:
                        prod.loc[0:t,v] = 0
                        prod.loc[t+delt,v] = prod.loc[t,v]-sol[k]/mu*cell[t]*(1-np.exp(mu*delt))
        else:
            #break
            cell[t+delt] = cell[t]
            for k, v in subs_ext.items():
                subs.loc[t+delt,v] = subs.loc[t,v]
            for k, v in prod_ext.items():
                prod.loc[t+delt,v] = prod.loc[t,v]

cell = cell[0:t]
subs = subs.loc[0:t]
prod = prod.loc[0:t, prod.columns[prod.max() > 0.1]]

### Plot cell mass and concentration profiles

In [ ]:
fig, ax = plt.subplots(figsize=(12,4), ncols=3, nrows=1)
cell.plot(ax=ax[0], style='s-', title='Cell', label='dcw', legend=True)
ax[0].set_xlabel("Hour")
ax[0].set_ylabel("Concentration (gDW/L)")
subs.plot(ax=ax[1], style='o-', title='Substrates')
ax[1].set_xlabel("Hour")
ax[1].set_ylabel("Concentration (mM)")
prod.plot(ax=ax[2], style='o-', title='Products')
ax[2].set_xlabel("Hour")
ax[2].set_ylabel("Concentration (mM)")
plt.tight_layout()